In [2]:
import os, sys
!{sys.executable} -m pip install torch

Defaulting to user installation because normal site-packages is not writeable
     |███████████████████▋            | 461.2 MB 103.4 MB/s eta 0:00:03

IOPub data rate exceeded.
The Jupyter server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--ServerApp.iopub_data_rate_limit`.

Current values:
ServerApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
ServerApp.rate_limit_window=3.0 (secs)



     |████████████████████████████████| 750.6 MB 13 kB/s s eta 0:00:01


In [4]:
import os, sys
!{sys.executable} -m pip install torchvision

Defaulting to user installation because normal site-packages is not writeable
     |████████████████████████████████| 21.0 MB 11.0 MB/s eta 0:00:01


In [5]:
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import TensorDataset, Dataset, DataLoader

from torch.utils.data import Subset
from torchvision import datasets, models, transforms, utils
from torch.autograd import Variable
import torch.optim as optim
from torch.optim import lr_scheduler
import torch.backends.cudnn as cudnn
cuda_enabled = torch.cuda.is_available()
# Device configuration
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
#cuda_enabled = False
CUDA_VISIBLE_DEVICES=0,1,2,3
cudnn.benchmark = True

from skimage import io, transform
import matplotlib.pyplot as plt

import os
import pandas as pd
import numpy as np
import glob
from __future__ import print_function, division
import datetime
import sys
from sklearn.metrics import accuracy_score, confusion_matrix
from sklearn.model_selection import train_test_split

from ast import literal_eval

# Ignore warnings
import warnings
warnings.filterwarnings("ignore")

plt.ion()   # interactive mode

In [6]:
class BiRNN(nn.Module):
    def __init__(self, input_size, hidden_size, num_layers, num_classes):
        super(BiRNN, self).__init__()
        self.is_training = False
        self.hidden_size = hidden_size
        self.num_layers = num_layers
        self.num_classes = num_classes
        self.lstm = nn.LSTM(input_size, hidden_size, num_layers, batch_first=True, bidirectional=True)
        self.fc = nn.Dropout(p=0.5, inplace=False)
        self.fc = nn.Linear(hidden_size*2, num_classes)
        self.linear = nn.Linear(self.hidden_size*2, self.num_classes)
        if cuda_enabled:
            self.lstm = self.lstm.cuda()
            self.fc = self.fc.cuda()
            self.linear = self.linear.cuda()
    
    def forward(self, x):
        # Set initial states
        h0 = Variable(torch.zeros(self.num_layers*2, x.size(0), self.hidden_size)) # 2 for bidirection 
        c0 = Variable(torch.zeros(self.num_layers*2, x.size(0), self.hidden_size))
        if cuda_enabled:
            h0 = h0.cuda()  # 2 for bidirection
            c0 = c0.cuda()
        # Forward propagate LSTM
        out, _ = self.lstm(x, (h0, c0))  # out: tensor of shape (batch_size, seq_length, hidden_size*2)
        
        # Decode hidden state of last time step
        if self.is_training:
            out = self.fc(out[:, -1, :])
        else:
            out = out[:, -1, :]
        # out = F.log_softmax(self.linear(out), dim=1)
        return out

In [10]:
class EyeLandmarksDataset(Dataset):
    """Face Landmarks dataset."""

    def __init__(self, csv_file, transform=None):#root_dir = img_dir
        """
        Args:
            csv_file (string): Path to the csv file with annotations.
            root_dir (string): Directory with all the images.
            transform (callable, optional): Optional transform to be applied
                on a sample.
        """
        self.landmarks_frame = pd.read_csv(csv_file)
        self.transform = transform

    def __len__(self):
        return len(self.landmarks_frame)#same as image label

    def __getitem__(self, idx):
        if torch.is_tensor(idx):
            idx = idx.tolist()

        # name = os.path.join(self.root_dir, self.landmarks_frame.iloc[idx, 0])
        name = self.landmarks_frame.iloc[idx, 0]
        labels = np.zeros(8)
        for i in range(8):
            labels[i]=name
        labels = torch.Tensor(labels).long()
        landmarks = self.landmarks_frame.iloc[idx, 1:]
        marks = []
        for mark in range(len(landmarks)):
            arr = literal_eval(landmarks[mark])[0]
            arr1 = []
            for i in range(len(arr)):
                for j in range(len(arr[i])):
                    # print(arr[i][j])
                    arr1.append(arr[i][j])
            marks.append(np.array(arr1))  
            # return
                          
        marks = torch.Tensor(marks)
        
        sample = {'label': labels, 'marks': marks}
        if self.transform:
            sample = self.transform(sample)
        return sample

In [11]:
eyedataset = EyeLandmarksDataset(csv_file='CSVs/EyeTracking.csv')
eyedataset.__len__(), eyedataset.__getitem__(0)['label'].shape, eyedataset.__getitem__(0)['marks'].shape

(224, torch.Size([8]), torch.Size([125, 8]))

In [12]:
def train_total_dataset(dataset, total_split=0.30): #Training set be 70% of the dataset
    train_idx, total_idx = train_test_split(list(range(len(dataset))), test_size=total_split)
    datasets = {}
    datasets['train'] = Subset(dataset, train_idx)
    datasets['total_split'] = Subset(dataset, total_idx)
    return datasets

# hyperparams for the network
batch_size = 64

print('Starting Eye NN dataset -------------------------------------')
eye_datasets = train_total_dataset(eyedataset)
eye_train = eye_datasets['train']
eye_test = eye_datasets['total_split']
# The original dataset is available in the Subset class
print(len(eye_datasets['train'].dataset), eye_datasets['train'].dataset)
print(len(eye_train), eye_train)
print(len(eye_test), eye_test)

print('Starting Eye NN loader -------------------------------------')
eye_train_loader = DataLoader(dataset=eye_train, batch_size=batch_size, shuffle=True)
eye_test_loader = DataLoader(dataset=eye_test, batch_size=batch_size, shuffle=False)
print(len(eye_train_loader), eye_train_loader)
print(len(eye_test_loader), eye_test_loader)
print('Finish  -------------------------------------')

Starting Eye NN dataset -------------------------------------
224 <__main__.EyeLandmarksDataset object at 0x1554c1056fd0>
156 <torch.utils.data.dataset.Subset object at 0x1554c1e7f5b0>
68 <torch.utils.data.dataset.Subset object at 0x1554c1e7f400>
Starting Eye NN loader -------------------------------------
3 <torch.utils.data.dataloader.DataLoader object at 0x1554c1055be0>
2 <torch.utils.data.dataloader.DataLoader object at 0x1554c1055b80>
Finish  -------------------------------------


In [18]:
#Init NN
input_size = 1
sequence_length = 125
hidden_size = 128
num_layers = 2
num_classes = 2  # TODO: Determine this from the data
learning_rate = 0.0001
num_epochs = 300

# The network
eye_model = BiRNN(input_size, hidden_size, num_layers, num_classes).to(device)
eye_model.is_training = True

# Loss and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(eye_model.parameters(), lr=learning_rate)

epoch_loss = 5000000000.

In [14]:
timing = dict()
timing['training'] = datetime.datetime.now()

In [ ]:
epochs = []
losses = []

for epoch in range(num_epochs):
    loss_total = 0.
    iteration_count = 0.
    for i, sample in enumerate(eye_train):
        iteration_count += 1.
        labels, marks = sample['label'], sample['marks'] 
        marks = Variable(marks.view(-1, sequence_length, input_size))
        labels = Variable(labels)
        if cuda_enabled:
            marks = marks.cuda()
            labels = labels.cuda()

        # Forward + Backward + Optimize
        optimizer.zero_grad()
        outputs = eye_model(marks)

        loss = criterion(outputs, labels)
        loss_total += loss.item()
        loss.backward()
        optimizer.step()

        if (i + 1) % 10 == 0:
            print('Epoch [%d/%d], Step [%d/%d], Loss: %.4f'
                  % (epoch + 1, num_epochs, i + 1, len(eye_train) // batch_size, loss.item()))
    current_epoch_loss = loss_total / iteration_count
    print('Epoch %d; loss = %0.4f' % (epoch, current_epoch_loss))
    epochs.append(epoch)
    losses.append(current_epoch_loss)
    epoch_loss = current_epoch_loss
timing['training'] = datetime.datetime.now() - timing['training']

Epoch [1/300], Step [10/2], Loss: 0.6735
Epoch [1/300], Step [20/2], Loss: 0.7418
Epoch [1/300], Step [30/2], Loss: 0.7058
Epoch [1/300], Step [40/2], Loss: 0.6305
Epoch [1/300], Step [50/2], Loss: 0.6780
Epoch [1/300], Step [60/2], Loss: 0.7269
Epoch [1/300], Step [70/2], Loss: 0.7840
Epoch [1/300], Step [80/2], Loss: 0.8624
Epoch [1/300], Step [90/2], Loss: 0.7261
Epoch [1/300], Step [100/2], Loss: 0.5959
Epoch [1/300], Step [110/2], Loss: 0.5799
Epoch [1/300], Step [120/2], Loss: 0.6393
Epoch [1/300], Step [130/2], Loss: 0.5594
Epoch [1/300], Step [140/2], Loss: 0.4643
Epoch [1/300], Step [150/2], Loss: 0.6321
Epoch 0; loss = 0.6587
Epoch [2/300], Step [10/2], Loss: 0.2611
Epoch [2/300], Step [20/2], Loss: 1.1266
Epoch [2/300], Step [30/2], Loss: 0.7711
Epoch [2/300], Step [40/2], Loss: 0.4734
Epoch [2/300], Step [50/2], Loss: 0.6314
Epoch [2/300], Step [60/2], Loss: 0.8018
Epoch [2/300], Step [70/2], Loss: 0.8779
Epoch [2/300], Step [80/2], Loss: 1.0136
Epoch [2/300], Step [90/2], 

In [ ]:
torch.save(eye_model.state_dict(), 'ShouldIDrive_eye_tracking.pkl')

In [ ]:
plt.plot(epochs, losses)

In [ ]:
eye_model.is_training = False
timing['testing'] = datetime.datetime.now()
print('Testing -----------------------------------------------')
correct = 0.0
total = 0.0
eye_predicted_list = []
eye_label_list = []
for i, sample in enumerate(eye_test_loader):#test_loader
    labels, marks = sample['label'], sample['marks'] 
    marks = Variable(marks.view(-1, sequence_length, input_size))
    if cuda_enabled:
        marks = marks.cuda()

    outputs = eye_model(marks)
    _, predicted = torch.max(outputs.data, 1)
    total += labels.size(0)
    # print(total)
    for p, l in zip(_, labels):
        eye_p = int(np.round(p).item())
        eye_l = l[0].item()
        eye_predicted_list.append(eye_p)
        eye_label_list.append(eye_l)
        if(eye_p == eye_l):
            correct += 1.0
    print(correct)
print('Test Accuracy of the model on the {} test images: {} %'.format(total, 100 * correct / total)) 

timing['testing'] = datetime.datetime.now() - timing['testing']
print(timing['testing'])

In [ ]:
print('Confusion Matrix')
print('================')
print(confusion_matrix(eye_label_list, eye_predicted_list))
print('=============================================')
print('Accuracy = %0.4f' % (accuracy_score(eye_label_list, eye_predicted_list)))
print('=============================================')